<a href="https://colab.research.google.com/github/pps21april/PanScienceInnovationsProject/blob/main/Unit_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%pip install langchain_community
%pip install langchain_experimental
%pip install langchain-openai
%pip install langchainhub
%pip install chromadb
%pip install langchain
%pip install beautifulsoup4
%pip install langchain-groq

In [10]:
from google.colab import userdata


In [11]:
%pip install PyPDF2

In [12]:
import os
import openai
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_experimental.text_splitter import SemanticChunker
import sentence_transformers
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_groq import ChatGroq
import PyPDF2

In [13]:
from google.colab import userdata

In [14]:
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import sqlite3
import json

In [15]:
def upload_documents(documents_folder_path):
    file_names = os.listdir(documents_folder_path)
    documents = []
    for filename in file_names:
        text = ""
        file_path = os.path.join(documents_folder_path, filename)
        if filename.endswith(".pdf"):
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    text += page.extract_text()
                    formatted_text = text.replace("\n", " ")
                    text_splitter = SemanticChunker(OpenAIEmbeddings(api_key=userdata.get("OPENAI_API_KEY")))
                    chunks = text_splitter.split_text(formatted_text)
                    for chunk in chunks:
                        doc = Document(page_content=chunk, metadata={"source": filename})
                        documents.append(doc)

    return documents

In [16]:
documents = upload_documents("documents_folder")

In [17]:
documents[0:5]

[Document(metadata={'source': 'Transformers Research Paper.pdf'}, page_content='Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need Ashish Vaswani∗ Google Brain avaswani@google.comNoam Shazeer∗ Google Brain noam@google.comNiki Parmar∗ Google Research nikip@google.comJakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research llion@google.comAidan N. Gomez∗ † University of Toronto aidan@cs.toronto.eduŁukasz Kaiser∗ Google Brain lukaszkaiser@google.com Illia Polosukhin∗ ‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on atte

In [18]:
def store_embeddings_create_retriever(documents_folder_path):
    embedding_function = OpenAIEmbeddings(model="text-embedding-ada-002",
                                      openai_api_key=userdata.get("OPENAI_API_KEY"))
    documents = upload_documents(documents_folder_path)
    vector_store = Chroma.from_documents(documents = documents,embedding = embedding_function)
    retriever = vector_store.as_retriever()
    return retriever

In [19]:
store_embeddings_create_retriever("documents_folder")

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f1ac16f52a0>, search_kwargs={})

In [20]:
def answering_user_queries(documents_folder_path,query):
    llm = ChatOpenAI(model_name="gpt-4o-mini", api_key=userdata.get("OPENAI_API_KEY"))
    retriever = store_embeddings_create_retriever(documents_folder_path)
    relevant_text = retriever.get_relevant_documents(query)

    prompt_template = """
    Answer user queries based on the context given in the documents below. Answers should be relevant,concise and relevant to the
    context . Return only the answer and nothing else. Always include sources of answer
    context : {context}
    query : {query}
    """

    prompt = PromptTemplate.from_template(prompt_template)
    rag_chain = prompt | llm
    response = rag_chain.invoke({"context":relevant_text,"query":query})
    return response.content

In [21]:
query = "What is a transformer"
documents_folder_path = "documents_folder"

answering_user_queries(documents_folder_path,query)

<ipython-input-20-0c6b836f94a9>:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_text = retriever.get_relevant_documents(query)


'A transformer is a new network architecture proposed for sequence transduction that relies solely on attention mechanisms, without using recurrence or convolutions. This model has demonstrated superior performance in machine translation tasks and shows great parallelizability, requiring significantly less training time compared to traditional models. (Source: Transformers Research Paper.pdf)'

In [22]:
def create_database_of_metadata(documents_folder_path,documents_folder_name):
    documents = upload_documents(documents_folder_path)

    # Connect to SQLite database (or create it)
    conn = sqlite3.connect("documents_metadata.db")
    cursor = conn.cursor()

    # Create table (if not exists)
    cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS {documents_folder_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        content TEXT,
        metadata TEXT
    )
    ''')

    # Insert data
    for doc in documents:
        cursor.execute(f"INSERT INTO {documents_folder_name} (content, metadata) VALUES (?, ?)",
                       (doc.page_content, json.dumps(doc.metadata)))

    # Commit and close
    conn.commit()
    conn.close()



In [23]:
create_database_of_metadata("documents_folder","documents_folder")

In [24]:
conn = sqlite3.connect("documents_metadata.db")
cursor = conn.cursor()
cursor.execute("select * from documents_folder")
rows = cursor.fetchall()
conn.close()
rows[0:5]

[(1,
  'Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need Ashish Vaswani∗ Google Brain avaswani@google.comNoam Shazeer∗ Google Brain noam@google.comNiki Parmar∗ Google Research nikip@google.comJakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research llion@google.comAidan N. Gomez∗ † University of Toronto aidan@cs.toronto.eduŁukasz Kaiser∗ Google Brain lukaszkaiser@google.com Illia Polosukhin∗ ‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. 